<a href="https://colab.research.google.com/github/AshwaniCoding/AshwaniCoding/blob/main/RPK_RAG_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Part 1 - Setup

In [1]:
# Install the required (1.37.0) versions to satisfy existing dependencies
# !pip uninstall opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 opentelemetry-proto==1.37.0 opentelemetry-exporter-otlp-proto-common==1.37.0 opentelemetry-exporter-otlp-proto-grpc==1.37.0 requests==2.32.5
!pip install -q opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 opentelemetry-proto==1.37.0 opentelemetry-exporter-otlp-proto-common==1.37.0 opentelemetry-exporter-otlp-proto-grpc==1.37.0 requests==2.32.5

In [2]:
!pip install -q -U google-generativeai langchain-google-genai langchain chromadb sentence-transformers langchain_cohere pypdf unstructured python-docx msoffcrypto-tool openpyxl

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Configure the API key
api_key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=api_key)

# Instantiate the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key)

print("✅ Gemini model configured successfully!")

✅ Gemini model configured successfully!


In [5]:
try:
    # Make a simple call to the Gemini model
    response = llm.invoke("Hello, Gemini!")

    # Print the response to confirm the connection
    print("✅ Gemini model connection successful!")
    print("Model response:", response.content)

except Exception as e:
    print(f"❌ Failed to connect to Gemini model: {e}")
    print("Please double-check your API key and permissions in Google Cloud Console.")

✅ Gemini model connection successful!
Model response: Hello! How can I help you today?


### Part 2 - Building the Baseline RAG System

##### Document Loading

In [6]:
# === Part 1.5 - New: Download URLs listed in url/urls.txt into data/external ===
import os
import requests
from pathlib import Path
from urllib.parse import urlparse

# Paths (according to your structure)

ROOT = Path.cwd()/ "drive/MyDrive/RPK RAG"

URLS_FILE = ROOT/ "url/urls.txt"
EXTERNAL_DIR = ROOT/ "data"

print("Project root:", ROOT)
print("Data dir exists:", EXTERNAL_DIR.exists())
print("urls.txt exists:", URLS_FILE.exists())

os.makedirs(EXTERNAL_DIR, exist_ok=True)

BROWSER_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept": (
        "text/html,application/xhtml+xml,application/xml;"
        "q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache"
}

session = requests.Session()

def download_from_url(url: str, save_dir: str = EXTERNAL_DIR, timeout: int = 30):
    """Download a URL to save_dir, return saved file path or None."""
    try:
        resp = session.get(url, headers=BROWSER_HEADERS, timeout=timeout)
        resp.raise_for_status()
        parsed = urlparse(url)
        filename = os.path.basename(parsed.path)
        # fallback filename if missing
        if not filename or "." not in filename:
            base = filename if filename else f"file_{abs(hash(url))}"
            filename = base + ".html"

        file_path = os.path.join(save_dir, filename)
        with open(file_path, "wb") as f:
            f.write(resp.content)
        print(f"Downloaded: {url} -> {file_path}")
        return file_path
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

# Read and download
downloaded_files = []
if os.path.exists(URLS_FILE):
    with open(URLS_FILE, "r") as f:
        urls = [ln.strip() for ln in f.readlines() if ln.strip()]
    print(f"Found {len(urls)} URLs in {URLS_FILE}")
    for url in urls:
        p = download_from_url(url, EXTERNAL_DIR)
        if p:
            downloaded_files.append(p)
    print(f"Downloaded {len(downloaded_files)} files to {EXTERNAL_DIR}")
else:
    print(f"No urls.txt found at {URLS_FILE}. Skipping downloads.")


Project root: /content/drive/MyDrive/RPK RAG
Data dir exists: True
urls.txt exists: True
Found 2 URLs in /content/drive/MyDrive/RPK RAG/url/urls.txt
Downloaded: https://www.kingfisher.com/about-us/our-leadership -> /content/drive/MyDrive/RPK RAG/data/our-leadership.html
Downloaded: https://www.kingfisher.com/responsible-business -> /content/drive/MyDrive/RPK RAG/data/responsible-business.html
Downloaded 2 files to /content/drive/MyDrive/RPK RAG/data


In [7]:
# --- Document loaders (support PDF, DOCX, XLSX, HTML, TXT) ---
from langchain.docstore.document import Document
import glob

# Prefer community loaders that handle unstructured content where possible.
# We'll attempt to use PyPDFLoader / UnstructuredWordDocumentLoader, and fall back cleanly.
from langchain_community.document_loaders import (
    PyPDFLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredExcelLoader,
    UnstructuredHTMLLoader,
)
from langchain.document_loaders import TextLoader

def load_file(path: str):
    """Load a file and return a list of Documents (0..n)."""
    try:
        lower = path.lower()
        if lower.endswith(".pdf"):
            return PyPDFLoader(path).load()
        elif lower.endswith(".docx") or lower.endswith(".doc"):
            return UnstructuredWordDocumentLoader(path).load()
        elif lower.endswith(".xlsx") or lower.endswith(".xls"):
            # Excel loader may return multiple docs (one per sheet or combined)
            return UnstructuredExcelLoader(path).load()
        elif lower.endswith(".html") or lower.endswith(".htm"):
            return UnstructuredHTMLLoader(path).load()
        elif lower.endswith(".txt"):
            return TextLoader(path, encoding="utf-8").load()
        else:
            # But skip binary-looking files
            print(f"Skipping unsupported file type: {path}")
            return []

        for d in docs:
          d.metadata = d.metadata or {}
          d.metadata["document_name"] = os.path.basename(path)
          d.metadata["source"] = path

        return docs

    except Exception as e:
        print(f"Error loading {path}: {e}")
        return []

def load_all_documents(data_folder="data"):
    """Walk data_folder and load supported documents; returns list[Document]."""
    docs = []
    for root, dirs, files in os.walk(data_folder):
        for fn in files:
            full = os.path.join(root, fn)
            # skip hidden/temp files
            if fn.startswith(".") or fn.endswith("~"):
                continue
            loaded = load_file(full)
            if loaded:
                docs.extend(loaded)
                print(f"Loaded {len(loaded)} doc(s) from {full}")
    print(f"Total documents loaded from {data_folder}: {len(docs)}")
    return docs

# Load everything (this replaces the single sample 10-K file)
all_documents = load_all_documents(EXTERNAL_DIR)

# If there were no documents loaded, as a fallback try the original single-file path if present
if not all_documents:
    raise RuntimeError(
        "No documents found in 'data/'. Please add files (pdf/docx/txt/html/xlsx) "
        "or check that downloads from url/urls.txt succeeded into data/external/."
    )

Loaded 5 doc(s) from /content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/KF_History.docx
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/FAQs at B&Q.html
Loaded 7 doc(s) from /content/drive/MyDrive/RPK RAG/data/Our business model – Kingfisher plc.pdf
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/Banners Details.xlsx
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/Own Exclusive Brands.docx
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/our-leadership.html
Loaded 1 doc(s) from /content/drive/MyDrive/RPK RAG/data/responsible-business.html
Total documents loaded from /content/drive/MyDrive/RPK RAG/data: 18


##### Chunking

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1200
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", " ", ""]
)

# Split documents into chunks (langchain expects split_documents for Document objects)
if all_documents:
    print("Splitting documents into chunks...")
    chunks = text_splitter.split_documents(all_documents)
    print(f"✅ Total chunks created: {len(chunks)}")
else:
    chunks = []
    print("No documents to split.")

Splitting documents into chunks...
✅ Total chunks created: 66


##### Vector Store

In [9]:
# === Embeddings + Vector Store (forced metadata preservation) =======================

from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_function = SentenceTransformerEmbeddings(model_name=embed_model_name)

persist_directory = str(ROOT / "chroma_store")

# Extract text + metadata manually
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks]

print(f"Total chunks: {len(chunks)}")
print(f"Example chunk metadata (first 3): {metadatas[:3]}")

# Force metadata into Chroma
vectordb = Chroma.from_texts(
    texts=texts,
    embedding=embedding_function,
    metadatas=metadatas,
    persist_directory=persist_directory
)

vectordb.persist()

print("✅ Vector DB created with metadata preserved.")

res = vectordb.similarity_search("kingfisher", k=1)
print(res[0].metadata)

/tmp/ipython-input-2513895487.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name=embed_model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets

Total chunks: 66
Example chunk metadata (first 3): [{'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': 'Kingfisher plc – Retail Market Overview', 'author': 'ChatGPT Deep Research', 'source': '/content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, {'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': 'Kingfisher plc – Retail Market Overview', 'author': 'ChatGPT Deep Research', 'source': '/content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, {'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': 'Kingfisher plc – Retail Market Overview', 'author': 'ChatGPT Deep Research', 'source': '/content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}]
✅ Vector DB created with metadata prese

/tmp/ipython-input-2513895487.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


##### QA Chain

In [10]:
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"document_variable_name": "context"}
)

print("✅ QA chain ready!\n")

✅ QA chain ready!



##### Initial Test

In [11]:
query = "How many banners are covered in Kingfisher?"

result = qa_chain.invoke({"query": query})

# Display the answer
print("=== 💬 MODEL ANSWER ===")
print(result["result"])
print("\n")

# Show the source snippets used by the model
print("=== 📄 SOURCE DOCUMENTS ===")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n--- Source {i} ---")
    print("Text:",doc.page_content[:400].replace("\n", " "), "...")
    print("Path:", doc.metadata.get("source"))
    print("📁 File:", doc.metadata.get("title"))

=== 💬 MODEL ANSWER ===
Kingfisher operates under 5 main banners:

1.  **B&Q** (UK & Ireland)
2.  **Screwfix** (UK & Ireland)
3.  **Castorama** (France & Poland)
4.  **Brico Dépôt** (France & Spain/Portugal)
5.  **Koçtaş** (Turkey)


=== 📄 SOURCE DOCUMENTS ===

--- Source 1 ---
Text: brick-and-mortar DIY. Kingfisher’s investment in e-commerce and retail media is partly to counter these shifts.  Geographic Reach Kingfisher’s footprint spans 7 European markets (UK, Ireland, France, Poland, Spain, Portugal, Turkey) . Major geographies by sales/store count: UK & Ireland (B&Q, Screwfix), France (Castorama, Brico Dépôt), Poland (Castorama), Spain/Portugal (Brico Dépôt), and Turkey ( ...
Path: /content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf
📁 File: Kingfisher plc – Retail Market Overview

--- Source 2 ---
Text: Kingfisher's Own Exclusive Brands (OEB) are a core component of its strategy ("Powered by Kingfisher"), driving differentiation across its retail banners

### Part 3 - Evaluating the Baseline

##### Creating a test set

In [12]:
test_questions = [
    "What is the primary product focus of Kingfisher plc?",
    "What is the name of Kingfisher plc's largest UK brand?",
    "In what year was Kingfisher plc officially established?",
    "Summarize the main goals of Kingfisher's 'ONE Kingfisher' strategy introduced in 2016."
]

##### Qualitative Evaluation

In [13]:
for i, q in enumerate(test_questions, 1):
    print(f"\n❓ Question {i}: {q}\n{'-'*80}")
    result = qa_chain.invoke(q)

    # Display answer
    print(f"💬 **Answer:**\n{result['result']}\n")

    # Display retrieved source chunks
    print("📚 **Retrieved Source Chunks:**")
    for idx, doc in enumerate(result["source_documents"], 1):
      print("Text:",doc.page_content[:400].replace("\n", " "), "...")
      print("Path:", doc.metadata.get("source"))
      print("📁 File:", doc.metadata.get("title"))



❓ Question 1: What is the primary product focus of Kingfisher plc?
--------------------------------------------------------------------------------
💬 **Answer:**
Kingfisher plc's primary product focus is **home improvement**.

They operate as a leading international home improvement retailer and their Own Exclusive Brands (OEB) portfolio covers a wide range of home improvement categories.

📚 **Retrieved Source Chunks:**
Text: schemes) who value fast delivery and specialist services. Competitors Kingfisher faces strong competition from global and local DIY retailers:  Adeo (Leroy Merlin group): Europe’s largest DIY retailer (39% French market share). Leroy Merlin leads in France, Spain, Poland etc., often with Castorama and Brico Dépôt close behind. Kingfisher executives note Adeo and Kingfisher together “dominate” the  ...
Path: /content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf
📁 File: Kingfisher plc – Retail Market Overview
Text: Kingfisher plc – Retail 

### Part 4 - Implementing an Advanced RAG Technique

##### Reranker Implementation

In [14]:
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
import os
from google.colab import userdata

# Load API key from Colab secrets
cohere_api_key = userdata.get("COHERE_API_KEY")
os.environ["COHERE_API_KEY"] = cohere_api_key

# Initialize the reranker
reranker = CohereRerank(model="rerank-english-v3.0")

# Build retriever with reranker
base_retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 10})
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever,
)

print("✅ Reranker and retriever configured successfully.")

✅ Reranker and retriever configured successfully.


##### Advanced QA Chain with Reranker

In [15]:
from langchain.chains import RetrievalQA

qa_chain_advanced = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True
)

print("✅ Advanced QA chain with reranker ready!\n")

✅ Advanced QA chain with reranker ready!



##### Re-Evaluation

In [16]:
print("=== 🔍 Re-evaluating Advanced RAG System ===\n")

for i, q in enumerate(test_questions, 1):
    print(f"\n❓ Question {i}: {q}\n{'-'*80}")
    result = qa_chain_advanced.invoke(q)

    # Display answer
    print(f"💬 **Answer:**\n{result['result']}\n")

    # Display retrieved source chunks
    print("📚 **Retrieved Source Chunks:**")
    for idx, doc in enumerate(result["source_documents"], 1):
      print("Text:",doc.page_content[:400].replace("\n", " "), "...")
      print("Path:", doc.metadata.get("source"))
      print("📁 File:", doc.metadata.get("title"))

=== 🔍 Re-evaluating Advanced RAG System ===


❓ Question 1: What is the primary product focus of Kingfisher plc?
--------------------------------------------------------------------------------
💬 **Answer:**
Kingfisher plc's primary product focus is **home improvement** and **DIY (Do It Yourself)** products. They are described as a leading international home improvement retailer and Europe's #2 DIY retailer.

📚 **Retrieved Source Chunks:**
Text: Kingfisher plc – Retail Market Overview Kingfisher plc is a leading international home improvement retailer (FTSE-100 listed) operating seven major  banners  (B&Q,  Screwfix,  Castorama,  Brico   Dépôt,  Koçtaş,  TradePoint)  across  Europe.  In FY2024/25 the Group reported ~£12.8bn in sales. Industry analyses highlight Kingfisher (net sales ~€13.8bn) as Europe’s #2 DIY retailer behind France’s Ad ...
Path: /content/drive/MyDrive/RPK RAG/data/Kingfisher plc – Retail Market Overview.pdf
📁 File: Kingfisher plc – Retail Market Overview
Text: The h

### Part 5 - Conclusion

Adding the reranker clearly improved the results, especially for Question 2 (executives) and Question 3 (competition risks).
The reranker helped reorder retrieved chunks by relevance, so the model focused on the most accurate and context-rich passages instead of random or repetitive ones.
This led to more precise answers, reduced “no information found” cases, and better context matching for fact-based queries.

However, in cases where the relevant data was not present in the retrieved context (like financial performance), the reranker couldn't help — it can only improve ranking, not fill in missing data.